In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-215146
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-215146


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "my-compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

vm_size = "Standard_D2_V2"
max_nodes = 4

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, using it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": choice(0.001, 0.01, 0.1, 1, 10, 100, 250, 1000), "--max_iter": choice(25, 50, 100, 200, 500)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', 
                script='train.py', 
                compute_target=cluster_name,
                environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=30,
max_concurrent_runs=4
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a
Web View: https://ml.azure.com/runs/HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-215146/workspaces/quick-starts-ws-215146&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-11-13T00:19:23.524335][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-11-13T00:19:24.4155290Z][SCHEDULER][INFO]Scheduling job, id='HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a_0' 
[2022-11-13T00:19:24.4715028Z][SCHEDULER][INFO]Scheduling job, id='HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a_1' 
[2022-11-13T00:19:24.6173947Z][SCHEDULER][INFO]Scheduling job, id='HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a_2' 
[2022-11-13T00:19:24.7328538Z][SCHEDULER][INFO]Scheduling job, id='HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a_3' 
[2022-11-13T00:19:24.699390][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a',
 'target': 'my-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-11-13T00:19:22.914069Z',
 'endTimeUtc': '2022-11-13T00:34:28.869573Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1b475b14-1cca-41d7-807a-deeb90fe948c',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '40',
  'score': '0.9088012139605463',
  'best_child_run_id': 'HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a_26',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a_26'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'a

In [6]:
import joblib
import pickle

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Details
print(best_run.get_details())

# metrics
print(best_run.get_metrics())

# files
print(best_run.get_file_names())

# Save model
best_run.register_model('hyperdrive-best-model.joblib', '.')

joblib.dump(best_run, 'hyperdrive-best-model.joblib')


{'runId': 'HD_7b4e51ee-f06a-44e1-b252-d28eecf73b0a_26', 'target': 'my-compute-cluster', 'status': 'Completed', 'startTimeUtc': '2022-11-13T00:31:09.92339Z', 'endTimeUtc': '2022-11-13T00:31:33.754033Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '1b475b14-1cca-41d7-807a-deeb90fe948c', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '100', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'my-compute-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'sklearn-env', 

Model(workspace=Workspace.create(name='quick-starts-ws-215146', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-215146'), name=hyperdrive-best-model.joblib, id=hyperdrive-best-model.joblib:8, version=8, tags={}, properties={})

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)


In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [10]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=False)
automl_run.wait_for_completion()

2022-11-13:01:08:58,688 WARNING  [local_experiment_driver.py:145] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_a2ce03e3-718e-4fc3-8d08-c580a7d173cb,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-11-13:01:42:55,110 INFO     [explanation_client.py:334] Using default datastore for uploads


{'runId': 'AutoML_a2ce03e3-718e-4fc3-8d08-c580a7d173cb',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-11-13T01:09:03.372881Z',
 'endTimeUtc': '2022-11-13T01:41:24.306152Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-215146","workspace_name":"quick-starts-ws-215146","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,

In [11]:
# Retrieve and save your best automl model.

best_automl_run, best_automl_model = automl_run.get_output()

best_automl_run.register_model('automl-best-model.pkl','.')
joblib.dump(best_automl_model, 'automl-best-model.joblib')


['automl-best-model.joblib']

In [12]:
# Cluster cleanup
compute_target.delete()


Current provisioning state of AmlCompute is "Deleting"

